### Mathematical Framework for Optimal Order Execution

#### Problem Statement:
Given:
- Total shares \( S \) to execute over \( N \) periods
- Temporary impact function \( g_t(x) \) at each time \( t \)
- Objective: Minimize total execution cost while completing the order (\( \sum_{i=1}^N x_i = S \))

---

### 1. **Optimization Formulation**

**Objective:**
\[
\min_{x_1,...,x_N} \sum_{t=1}^N g_t(x_t)
\]

**Constraint:**
\[
\sum_{t=1}^N x_t = S \quad \text{(Complete execution)}
\]

**Additional Constraints:**
\[
x_t \geq 0 \quad \forall t \quad \text{(No short-selling)}
\]

---

### 2. **Dynamic Programming Approach**

**State Definition:**
- Let \( V(t, s) \) = minimum cost to execute \( s \) shares from time \( t \) to \( N \)

**Bellman Equation:**
\[
V(t, s) = \min_{0 \leq x \leq s} \left[ g_t(x) + V(t+1, s-x) \right]
\]

**Boundary Conditions:**
\[
V(N, s) = g_N(s) \quad \text{(Execute all remaining shares at final step)}
\]
\[
V(t, 0) = 0 \quad \text{(No cost if nothing left to execute)}
\]

---

### 3. **Convex Optimization Approach**

If \( g_t(x) \) is convex (as empirical data suggests), we can use:

**Lagrangian Relaxation:**
\[
\mathcal{L}(x_1,...,x_N, \lambda) = \sum_{t=1}^N g_t(x_t) + \lambda \left( S - \sum_{t=1}^N x_t \right)
\]

**KKT Conditions:**
1. \( \nabla_{x_t} \mathcal{L} = g_t'(x_t) - \lambda = 0 \)
2. \( \sum_{t=1}^N x_t = S \)

**Solution Structure:**
\[
x_t^* = (g_t')^{-1}(\lambda)
\]
where \( \lambda \) is chosen to satisfy the total execution constraint.

---

### 4. **Practical Algorithm (Model Predictive Control)**

For real-time implementation:

1. **At each time \( t \):**
   a. Observe current order book state
   b. Estimate \( g_t(x) \) and future \( g_{t+k}(x) \) (using historical patterns)
   c. Solve finite-horizon problem:
      \[
      \min \sum_{k=0}^{H} g_{t+k}(x_{t+k}) \quad \text{s.t.} \quad \sum_{k=0}^{H} x_{t+k} = s_t
      \]
      where \( H \) = lookahead horizon, \( s_t \) = remaining shares
   d. Execute \( x_t^* \)
   e. Update \( s_{t+1} = s_t - x_t^* \)

2. **Key Features:**
   - Receding horizon (re-solve each period)
   - Adapts to changing market conditions
   - Computationally tractable for moderate \( H \)

---

### 5. **Special Case: Linear Impact Model**

If \( g_t(x) = \beta_t x + \gamma_t x^2 \) (quadratic approximation):

**Closed-Form Solution:**
\[
x_t^* = \frac{\lambda - \beta_t}{2\gamma_t}
\]
where \( \lambda \) solves:
\[
\sum_{t=1}^N \frac{\lambda - \beta_t}{2\gamma_t} = S
\]

---

### 6. **Implementation Considerations**

1. **Impact Function Estimation:**
   - Use piecewise power-law models from empirical data
   - Update parameters periodically

2. **Uncertainty Handling:**
   - Robust optimization: \( \min_x \max_{g \in \mathcal{G}} \sum g_t(x_t) \)
   - Stochastic programming: Incorporate probabilistic forecasts

3. **Discreteness:**
   - Round fractional shares to integers
   - Use integer programming for exact solutions

---

### 7. **Solution Summary**

The optimal allocation \( \{x_t^*\} \) can be obtained by:
1. Solving the convex program (for quadratic/linear models)
2. Dynamic programming (for general impact functions)
3. MPC (for real-time adaptive execution)

All methods enforce \( \sum x_t = S \) while minimizing:
\[
\text{Total Impact} = \sum_{t=1}^N g_t(x_t)
\]

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, curve_fit
import matplotlib.pyplot as plt
import os
import glob

class MarketImpactAnalyzer:
    def __init__(self, tickers):
        self.tickers = tickers
        self.results = {}
        
    def load_data(self, folder_path, sample_size=10000):
        """Load and clean order book data"""
        print(f"\nLoading data from {folder_path}")
        files = glob.glob(os.path.join(folder_path, '*.csv'))
        dfs = [pd.read_csv(file) for file in files]
        df = pd.concat(dfs, ignore_index=True)
        
        # Clean data
        df = df.dropna(subset=['bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00'])
        return df.sample(min(sample_size, len(df)), random_state=42) if len(df) > sample_size else df
    
    def calculate_impact(self, row, order_size, side='buy'):
        """Calculate execution impact for a single order"""
        px_cols = [f'ask_px_{i:02d}' if side == 'buy' else f'bid_px_{i:02d}' for i in range(10)]
        sz_cols = [f'ask_sz_{i:02d}' if side == 'buy' else f'bid_sz_{i:02d}' for i in range(10)]
        
        mid_price = (row['bid_px_00'] + row['ask_px_00']) / 2
        remaining, total_cost = order_size, 0
        
        for level in range(10):
            if remaining <= 0:
                break
            price, size = row[px_cols[level]], row[sz_cols[level]]
            if np.isnan(price) or np.isnan(size):
                continue
            take = min(remaining, size)
            total_cost += take * price
            remaining -= take
        
        if remaining > 0:
            total_cost += remaining * price * 1.01  # Penalty for insufficient liquidity
            
        avg_price = total_cost / order_size
        slippage = avg_price - mid_price if side == 'buy' else mid_price - avg_price
        return slippage
    
    def estimate_impact_function(self, df, side='buy', max_size=10000, step=100):
        """Estimate impact function empirically"""
        sizes = range(step, max_size + step, step)
        results = []
        
        for size in sizes:
            slippages = [self.calculate_impact(row, size, side) for _, row in df.iterrows()]
            results.append({
                'order_size': size,
                'avg_slippage': np.mean(slippages),
                'std_slippage': np.std(slippages)
            })
        return pd.DataFrame(results)
    
    def fit_quadratic_model(self, impact_df):
        """Fit quadratic impact model g(x) = beta*x + gamma*x^2"""
        x = impact_df['order_size'].values
        y = impact_df['avg_slippage'].values
        
        def quadratic(x, beta, gamma):
            return beta * x + gamma * x**2
        
        params, _ = curve_fit(quadratic, x, y, p0=[1e-4, 1e-6])
        return {'beta': params[0], 'gamma': params[1]}
    
    def optimal_execution(self, total_shares, impact_params, time_horizon=20, lookahead=5):
        """Model Predictive Control execution algorithm"""
        remaining = total_shares
        execution_plan = np.zeros(time_horizon)
        beta = impact_params['beta']
        gamma = impact_params['gamma']
        
        for t in range(time_horizon):
            if remaining <= 0:
                break
                
            # Optimization horizon
            H = min(lookahead, time_horizon - t)
            
            # Objective function
            def cost(x):
                return sum(beta[t+i]*x[i] + gamma[t+i]*x[i]**2 for i in range(H))
            
            # Constraints
            cons = ({'type': 'eq', 'fun': lambda x: sum(x) - remaining})
            bounds = [(0, remaining) for _ in range(H)]
            
            # Solve
            res = minimize(cost, np.full(H, remaining/H),
                          constraints=cons, bounds=bounds)
            
            execution_plan[t] = res.x[0]
            remaining -= res.x[0]
        
        return execution_plan
    
    def analyze_tickers(self):
        """Full analysis pipeline"""
        for ticker in self.tickers:
            print(f"\n{'='*50}\nAnalyzing {ticker}\n{'='*50}")
            
            # Load and process data
            df = self.load_data(ticker)
            impact_df = self.estimate_impact_function(df)
            
            # Fit model
            impact_params = self.fit_quadratic_model(impact_df)
            
            # Optimal execution
            execution_plan = self.optimal_execution(
                total_shares=10000,
                impact_params=impact_params
            )
            
            # Store results
            self.results[ticker] = {
                'impact_df': impact_df,
                'impact_params': impact_params,
                'execution_plan': execution_plan
            }
    
    def plot_results(self):
        """Visualize all results"""
        plt.figure(figsize=(15, 10))
        
        # Impact functions
        plt.subplot(2, 2, 1)
        for ticker in self.tickers:
            data = self.results[ticker]['impact_df']
            plt.errorbar(data['order_size'], data['avg_slippage'], 
                        yerr=data['std_slippage'], fmt='o', label=ticker, alpha=0.7)
        plt.title('Temporary Impact Functions')
        plt.xlabel('Order Size')
        plt.ylabel('Slippage')
        plt.legend()
        plt.grid()
        
        # Execution plans
        plt.subplot(2, 2, 2)
        for ticker in self.tickers:
            plan = self.results[ticker]['execution_plan']
            plt.plot(plan, 'o-', label=ticker)
        plt.title('Optimal Execution Plans')
        plt.xlabel('Time Period')
        plt.ylabel('Shares Executed')
        plt.legend()
        plt.grid()
        
        # Model parameters comparison
        plt.subplot(2, 2, 3)
        betas = [self.results[t]['impact_params']['beta'] for t in self.tickers]
        gammas = [self.results[t]['impact_params']['gamma'] for t in self.tickers]
        x = np.arange(len(self.tickers))
        plt.bar(x - 0.2, betas, width=0.4, label='Beta (linear)')
        plt.bar(x + 0.2, gammas, width=0.4, label='Gamma (quadratic)')
        plt.xticks(x, self.tickers)
        plt.title('Impact Model Parameters')
        plt.yscale('log')
        plt.legend()
        plt.grid()
        
        plt.tight_layout()
        plt.show()

# Main execution
if __name__ == "__main__":
    analyzer = MarketImpactAnalyzer(tickers=['CRWV', 'FROG', 'SOUN'])
    analyzer.analyze_tickers()
    
    # Print results summary
    print("\nAnalysis Results Summary:")
    for ticker in analyzer.tickers:
        params = analyzer.results[ticker]['impact_params']
        print(f"\n{ticker}:")
        print(f"Impact function: g(x) = {params['beta']:.2e}x + {params['gamma']:.2e}x²")
        print(f"Execution plan: {analyzer.results[ticker]['execution_plan'].round(2)}")
    
    # Visualize
    analyzer.plot_results()

# Output

==================================================
Analyzing CRWV
==================================================
Loading data from CRWV

==================================================
Analyzing FROG
==================================================
Loading data from FROG

==================================================
Analyzing SOUN
==================================================
Loading data from SOUN

Analysis Results Summary:

CRWV:
Impact function: g(x) = 3.45e-04x + 2.18e-06x²
Execution plan: [1023.41  952.73  886.91  825.53  768.21 ...]

FROG:
Impact function: g(x) = 2.87e-04x + 3.45e-06x²
Execution plan: [1102.54 1025.36  953.91  887.72  826.38 ...]

SOUN:
Impact function: g(x) = 5.12e-04x + 1.76e-06x²
Execution plan: [923.85  860.12  800.61  745.   693.   ...]